In [2]:
import requests
from bs4 import BeautifulSoup
from JRDBDataParsingTools.data.weather import get_session_id

# Station

In [91]:
import json


# class Prefecture:
#     def __init__(self, name, id):
#         self.name = name
#         self.id = id

#     def __repr__(self):
#         return str(self)

#     def __str__(self):
#         return f"{self.name}: {self.id}"

#     def to_json(self):
#         return json.dumps({"id": self.id, "name": self.name}, indent=4)

req = requests.post(
    "https://www.data.jma.go.jp/gmd/risk/obsdl/top/station", data={"pd": "00"}
)
req.raise_for_status()
bs4 = BeautifulSoup(req.content, "html.parser")

for prefecture in bs4.find_all("div", {"class": "prefecture"}):
    name = prefecture.text
    id_ = prefecture.find("input", {"name": "prid"}).get("value")
    print(name, id_)

宗谷 11
留萌 13
上川 12
網走・北見・紋別 17
空知 15
後志 16
石狩 14
日高 22
十勝 20
根室 18
檜山 24
胆振 21
釧路 19
渡島 23
青森 31
秋田 32
岩手 33
石川 56
山形 35
宮城 34
富山 55
新潟 54
福島 36
山口 81
島根 68
鳥取 69
兵庫 63
京都 61
福井 57
岐阜 52
長野 48
群馬 42
栃木 41
茨城 40
広島 67
岡山 66
滋賀 60
山梨 49
埼玉 43
長崎 84
佐賀 85
福岡 82
大阪 62
奈良 64
三重 53
愛知 51
静岡 50
神奈川 46
東京 44
千葉 45
熊本 86
大分 83
愛媛 73
香川 72
和歌山 65
鹿児島 88
宮崎 87
高知 74
徳島 71
沖縄 91
南極 99


In [129]:
import re


def get_stations(prefecture_code):
    """
    prefecture_code: "45"
    """
    result = []

    req = requests.post(
        "https://www.data.jma.go.jp/gmd/risk/obsdl/top/station",
        data={"pd": prefecture_code},
    )
    req.raise_for_status()
    bs4 = BeautifulSoup(req.content, "html.parser")

    for station in bs4.find_all("div", {"class": "station"}):
        stid = station.find("input", {"name": "stid"}).get("value")
        stname = station.find("input", {"name": "stname"}).get("value")
        prid = station.find("input", {"name": "prid"}).get("value")

        kansoku_raw = station.find("input", {"name": "kansoku"}).get("value")
        kansoku = {
            "rain": kansoku_raw[0] == "1",
            "wind": kansoku_raw[1] == "1",
            "temp": kansoku_raw[2] == "1",
            "sun": kansoku_raw[3] == "1",
            "snow": kansoku_raw[4] == "1",
        }

        lat = re.search(r"北緯：(\d+)度(\d+\.\d+)分", station["title"]).groups()
        lat = float(lat[0]) + float(lat[1]) / 60

        lon = re.search(r"東経：(\d+)度(\d+\.\d+)分", station["title"]).groups()
        lon = float(lon[0]) + float(lon[1]) / 60

        alt = re.search(r"標高：(.+)m", station["title"]).groups()[0]
        alt = float(alt)

        kana = re.search(r"カナ:(.+)", station["title"]).groups()[0]

        result.append(
            {
                "station_code": stid,
                "station_name": stname,
                "prefecture_code": prid,
                "observations": kansoku,
                "latitude": lat,
                "longitude": lon,
                "station_name_kana": kana,
                "altitude_meters": alt,
            }
        )

    return result


get_stations(prefecture_code="45")

[{'station_code': 'a0375',
  'station_name': '香取',
  'prefecture_code': '45',
  'observations': {'rain': True,
   'wind': True,
   'temp': True,
   'sun': False,
   'snow': False},
  'latitude': 35.858333333333334,
  'longitude': 140.50166666666667,
  'station_name_kana': 'カトリ',
  'altitude_meters': 37.0},
 {'station_code': 'a0375',
  'station_name': '香取',
  'prefecture_code': '45',
  'observations': {'rain': True,
   'wind': True,
   'temp': True,
   'sun': False,
   'snow': False},
  'latitude': 35.858333333333334,
  'longitude': 140.50166666666667,
  'station_name_kana': 'カトリ',
  'altitude_meters': 37.0},
 {'station_code': 'a0376',
  'station_name': '我孫子',
  'prefecture_code': '45',
  'observations': {'rain': True,
   'wind': True,
   'temp': True,
   'sun': False,
   'snow': False},
  'latitude': 35.86333333333333,
  'longitude': 140.11,
  'station_name_kana': 'アビコ',
  'altitude_meters': 20.0},
 {'station_code': 'a0376',
  'station_name': '我孫子',
  'prefecture_code': '45',
  'observ

# Periods

In [83]:
# <div id="aggrgPeriod">
#     <h2>データの種類<span class="manual right"><a href="//www.data.jma.go.jp/risk/obsdl/top/help1.html#koumoku_select" style="color:#FFFFFF;">詳細</a>&nbsp;</span></h2>
#     <div class="clearfix">
#         <div>
#             <div>
#                 <label onclick="">
#                     <input type="radio" name="aggrgPeriod" value="9">
#                     <span>時別値</span>
#                 </label>
#             </div>
#             <div>
#                 <label onclick="">
#                     <input type="radio" name="aggrgPeriod" value="1">
#                     <span>日別値</span>
#                 </label>
#             </div>
#             <div>
#                 <input type="radio" name="aggrgPeriod" value="8">
#                 <span>
#                     <select class="inumber" name="nday">
#                         <option value="2">2</option>
#                         <option value="3">3</option>
#                         <option value="4">4</option>
#                         ...
#                     </select>
#                     <label onclick="" for="N日別値">日別値</label>
#                 </span>
#             </div>
#             <div>
#                 <label onclick="">
#                     <input type="radio" name="aggrgPeriod" value="2">
#                     <span>半旬別値</span>
#                 </label>
#             </div>
#             <div>
#                 <label onclick="">
#                     <input type="radio" name="aggrgPeriod" value="4">
#                     <span>旬別値</span>
#                 </label>
#             </div>
#             <div>
#                 <label onclick="">
#                     <input type="radio" name="aggrgPeriod" value="5">
#                     <span>月別値</span>
#                 </label>
#             </div>
#             <div>
#                 <label onclick="">
#                     <input type="radio" name="aggrgPeriod" value="6">
#                     <span>3か月別値</span>
#                 </label>
#                 <span class="green s">※</span>
#             </div>
#         </div>
#         <div>
#             <img src="//www.data.jma.go.jp/risk/obsdl/web/img/koumoku_set_rmk.png" class="" alt=""/>
#         </div>
#     </div>
# </div>


def get_periods():
    """
    Does not handle <select> tag
    O(n)
    """
    req = requests.get("https://www.data.jma.go.jp/gmd/risk/obsdl/top/element")
    soup = BeautifulSoup(req.content, "html.parser")
    result = {}
    for label in soup.find("div", {"id": "aggrgPeriod"}).find_all("label"):
        input_tag = label.find("input")
        span_tag = label.find("span")
        # for=N日別値
        # if label.get("for") == "N日別値":
        #     for select_tag in label.parent.find("select"):
        #         result[select_tag.text.strip()] = select_tag.get("value")
        if input_tag and span_tag:
            result[span_tag.text.strip()] = input_tag.get("value")
    return result


get_periods()

{'時別値': '9', '日別値': '1', '半旬別値': '2', '旬別値': '4', '月別値': '5', '3か月別値': '6'}

# Elements

In [85]:
requests.post(
    "https://www.data.jma.go.jp/gmd/risk/obsdl/top/element",
    data={"aggrgPeriod": 9, "prid": 1, "blockNo": 47662, "view": "", "preCnt": 1},
)

'<div class="clearfix"><input type="submit" id="buttonDelElement" value="ã\x81\x99ã\x81¹ã\x81¦ã\x81®é\x81¸æ\x8a\x9eæ¸\x88ã\x81¿ã\x81®é\xa0\x85ç\x9b®ã\x82\x92ã\x82¯ã\x83ªã\x82¢"><span class="manual right"><a href="//www.data.jma.go.jp/risk/obsdl/top/help1.html#koumoku_select">é\xa0\x85ç\x9b®é\x81¸æ\x8a\x9eã\x81®ä½¿ã\x81\x84æ\x96¹</a></span></div>\n<div class="clearfix" id="areaKoumokuTop">\n    <div id="aggrgPeriod"> \n        <h2>ã\x83\x87ã\x83¼ã\x82¿ã\x81®ç¨®é¡\x9e<span class="manual right"><a href="//www.data.jma.go.jp/risk/obsdl/top/help1.html#koumoku_select" style="color:#FFFFFF;">è©³ç´°</a>&nbsp;</span></h2>\n        <div class="clearfix">\n            <div>\n                <div><label onclick=""><input type="radio" name="aggrgPeriod" value="9"><span>æ\x99\x82å\x88¥å\x80¤</span></label></div>\n                <div><label onclick=""><input type="radio" name="aggrgPeriod" value="1"><span>æ\x97¥å\x88¥å\x80¤</span></label></div>\n                <div><input type="radio" name="aggrgPe

In [ ]:
# stationNumList=%5B%22a1236%22%5D
# aggrgPeriod=9
# elementNumList=%5B%5B%22201%22%2C%22%22%5D%2C%5B%22703%22%2C%22%22%5D%2C%5B%22101%22%2C%22%22%5D%2C%5B%22503%22%2C%22%22%5D%2C%5B%22501%22%2C%22%22%5D%2C%5B%22401%22%2C%22%22%5D%2C%5B%22301%22%2C%22%22%5D%2C%5B%22612%22%2C%22%22%5D%2C%5B%22604%22%2C%22%22%5D%2C%5B%22605%22%2C%22%22%5D%2C%5B%22602%22%2C%22%22%5D%2C%5B%22601%22%2C%22%22%5D%2C%5B%22607%22%2C%22%22%5D%2C%5B%22704%22%2C%22%22%5D%2C%5B%22610%22%2C%22%22%5D%5D
# interAnnualFlag=1
# ymdList=%5B%222023%22%2C%222023%22%2C%221%22%2C%221%22%2C%221%22%2C%2215%22%5D
# optionNumList=%5B%5D
# downloadFlag=true
# rmkFlag=1
# disconnectFlag=1
# youbiFlag=0
# fukenFlag=0
# kijiFlag=0
# huukouFlag=0
# csvFlag=1
# jikantaiFlag=0
# jikantaiList=%5B1%2C24%5D
# ymdLiteral=1
# PHPSESSID=a55l4n3v4e4g6kbmc5ioak96c7

params = {
    # "PHPSESSID": phpsessid,

    # 共通フラグ
    "rmkFlag": 1,  # 利用上注意が必要なデータを格納する
    "disconnectFlag": 1,  # 観測環境の変化にかかわらずデータを格納する
    "csvFlag": 1,  # すべて数値で格納する
    "ymdLiteral": 1,  # 日付は日付リテラルで格納する
    "youbiFlag": 0,  # 日付に曜日を表示する
    "kijiFlag": 0,  # 最高・最低（最大・最小）値の発生時刻を表示

    # 時別値データ選択
    "aggrgPeriod": 9,  # 時別値
    # "stationNumList": '["%s"]' % station,  # 観測地点IDのリスト
    # "elementNumList": '[["%s",""]]' % element,  # 項目IDのリスト
    # "ymdList": '["%d", "%d", "%d", "%d", "%d", "%d"]'
    # % (
    #     begin_date.year,
    #     end_date.year,
    #     begin_date.month,
    #     end_date.month,
    #     begin_date.day,
    #     end_date.day,
    # ),  # 取得する期間
    "jikantaiFlag": 0,  # 特定の時間帯のみ表示する
    "jikantaiList": "[1,24]",  # デフォルトは全部
    "interAnnualFlag": 1,  # 連続した期間で表示する

    # 以下、意味の分からないフラグ類
    "optionNumList": [],
    "downloadFlag": "true",  # CSV としてダウンロードする？
    "huukouFlag": 0,
}